In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

import tensorflow as tf


print(tf.__version__)

train_data_dir = pathlib.Path("./data/train")
val_data_dir = pathlib.Path("./data/validation")
test_data_dir = pathlib.Path("./data/test")


2.3.0


In [12]:
train_image_count = len(list(train_data_dir.glob('*/*.jpg')))
test_image_count = len(list(test_data_dir.glob('*/*.jpg')))
val_image_count = len(list(val_data_dir.glob('*/*.jpg')))

train_list_ds = tf.data.Dataset.list_files(str(train_data_dir/'*/*'))
val_list_ds = tf.data.Dataset.list_files(str(val_data_dir/'*/*'))
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir/'*/*'))

In [13]:
class_names = np.array([item.name for item in train_data_dir.glob('*')])
print(f"Class names: {class_names}")

#data augmentation function
IMG_WIDTH=100
IMG_HEIGHT=100

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def get_label(path):
  parts = tf.strings.split(path, os.path.sep)
  return parts[-2] == class_names[0]

def process_path(file_path):
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  label = get_label(file_path)
  return img, label

Class names: ['swift' 'wagonr']


In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#train_labeled_ds = train_labeled_ds.cache().shuffle()
#train_labeled_ds = train_labeled_ds.repeat()
train_ds = train_labeled_ds.cache().shuffle(buffer_size=1000).repeat().batch(25).prefetch(buffer_size=AUTOTUNE)
#train_labeled_ds = train_labeled_ds.prefetch(buffer_size=AUTOTUNE)

test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#test_labeled_ds = test_labeled_ds.cache()
test_ds = test_labeled_ds.cache().batch(25).prefetch(buffer_size=AUTOTUNE)
#test_labeled_ds = test_labeled_ds.prefetch(buffer_size=AUTOTUNE)

val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_labeled_ds.cache().batch(25).prefetch(buffer_size=AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [15]:
for image, label in train_labeled_ds.take(10):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  False
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  True
Image shape:  (100, 100, 3)
Label:  False


In [16]:
print(model.summary())



NameError: name 'model' is not defined

In [17]:
batch_size = 32
img_height = 100
img_width = 100
steps_per_epoch = np.ceil(train_image_count/batch_size)
input_shape = (100, 100, 3)
num_outputs = 1
num_epochs = 10


import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

tensorboard_callback = keras.callbacks.TensorBoard()

training_history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    callbacks=[tensorboard_callback])

print(f"Average test loss: {np.average(training_history.history['loss'])}")
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

print(f"\nTest accuracy: {test_acc}")
print(f"\nValidation accuracy: {val_acc}")


Epoch 1/10
 1/75 [..............................] - ETA: 0s - loss: 0.6704 - accuracy: 0.6800WARNING:tensorflow:From C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
75/75 [==============================] - ETA: 2s - loss: 5.2798 - accuracy: 0.50 - ETA: 2s - loss: 4.4973 - accuracy: 0.50 - ETA: 2s - loss: 4.0907 - accuracy: 0.51 - ETA: 2s - loss: 4.1637 - accuracy: 0.52 - ETA: 2s - loss: 3.8509 - accuracy: 0.52 - ETA: 2s - loss: 3.6071 - accuracy: 0.52 - ETA: 2s - loss: 3.3721 - accuracy: 0.53 - ETA: 2s - loss: 3.0947 - accuracy: 0.54 - ETA: 1s - loss: 3.0102 - accuracy: 0.54 - ETA: 1s - loss: 2.9058 - accuracy: 0.54 - ETA: 1s - loss: 2.8427 - accuracy: 0.54 - ETA: 1s - loss: 2.7647 - accuracy: 0.54 - ETA: 1s - loss: 2.6685 - accuracy: 0.54 - ETA: 1s - loss: 2.5849 - accur

75/75 [==============================] - ETA: 0s - loss: 0.7219 - accuracy: 0.68 - ETA: 0s - loss: 0.7139 - accuracy: 0.62 - ETA: 0s - loss: 0.7607 - accuracy: 0.60 - ETA: 0s - loss: 0.8192 - accuracy: 0.61 - ETA: 0s - loss: 0.8062 - accuracy: 0.62 - ETA: 0s - loss: 0.7711 - accuracy: 0.64 - ETA: 0s - loss: 0.7600 - accuracy: 0.64 - ETA: 0s - loss: 0.7315 - accuracy: 0.66 - ETA: 0s - loss: 0.6965 - accuracy: 0.67 - ETA: 0s - loss: 0.6740 - accuracy: 0.68 - ETA: 0s - loss: 0.6804 - accuracy: 0.68 - ETA: 0s - loss: 0.6888 - accuracy: 0.67 - ETA: 0s - loss: 0.6981 - accuracy: 0.67 - ETA: 0s - loss: 0.7381 - accuracy: 0.66 - ETA: 0s - loss: 0.7174 - accuracy: 0.66 - ETA: 0s - loss: 0.7022 - accuracy: 0.67 - ETA: 0s - loss: 0.6923 - accuracy: 0.67 - ETA: 0s - loss: 0.6834 - accuracy: 0.68 - ETA: 0s - loss: 0.6822 - accuracy: 0.67 - 1s 15ms/step - loss: 0.6787 - accuracy: 0.6805 - val_loss: 0.7178 - val_accuracy: 0.6175
Epoch 9/10
75/75 [==============================] - ETA: 0s - loss: 0.38

In [81]:
# Classifiers
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Metrics
from sklearn.metrics import precision_recall_fscore_support

In [98]:
%load_ext tensorboard

%tensorboard --logdir logs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 166540), started 0:00:53 ago. (Use '!kill 166540' to kill it.)

In [96]:
!kill 164724

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
from kerastuner.tuners import RandomSearch
import tensorflow as tf
from tensorflow import keras

input_shape = (100, 100, 3)
num_outputs = 1
num_epochs = 10

In [25]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(units=hp.Int('l1_units',
                                        min_value=32,
                                        max_value=512,
                                        step=32), activation='relu'),
        keras.layers.Dense(units=hp.Int('l2_units',
                                        min_value=32,
                                        max_value=512,
                                        step=32), activation='relu'),
        keras.layers.Dense(units=hp.Int('l3_units',
                                        min_value=32,
                                        max_value=512,
                                        step=32), activation='relu'),
        keras.layers.Dense(num_outputs)
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [26]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

INFO:tensorflow:Reloading Oracle from existing project my_dir\helloworld\oracle.json


In [27]:
tuner.search_space_summary()


In [ ]:
tuner.search(
    train_ds, 
    validation_data=val_ds, 
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch)

Epoch 1/10
75/75 [==============================] - ETA: 0s - loss: 0.7371 - accuracy: 0.60 - ETA: 0s - loss: 1.0845 - accuracy: 0.49 - ETA: 0s - loss: 0.9708 - accuracy: 0.48 - ETA: 0s - loss: 0.8824 - accuracy: 0.51 - ETA: 0s - loss: 0.8473 - accuracy: 0.50 - ETA: 0s - loss: 0.8195 - accuracy: 0.49 - ETA: 0s - loss: 0.8012 - accuracy: 0.50 - ETA: 0s - loss: 0.7895 - accuracy: 0.50 - ETA: 0s - loss: 0.7746 - accuracy: 0.50 - ETA: 0s - loss: 0.7761 - accuracy: 0.50 - ETA: 0s - loss: 0.7734 - accuracy: 0.50 - 1s 11ms/step - loss: 0.7721 - accuracy: 0.5083 - val_loss: 0.7128 - val_accuracy: 0.5000
Epoch 2/10
75/75 [==============================] - ETA: 0s - loss: 0.6601 - accuracy: 0.56 - ETA: 0s - loss: 0.7338 - accuracy: 0.49 - ETA: 0s - loss: 0.7372 - accuracy: 0.51 - ETA: 0s - loss: 0.7537 - accuracy: 0.51 - ETA: 0s - loss: 0.7429 - accuracy: 0.52 - ETA: 0s - loss: 0.7418 - accuracy: 0.54 - ETA: 0s - loss: 0.7400 - accuracy: 0.53 - ETA: 0s - loss: 0.7366 - accuracy: 0.54 - ETA: 0s -

75/75 [==============================] - ETA: 0s - loss: 0.6895 - accuracy: 0.48 - ETA: 0s - loss: 0.6712 - accuracy: 0.58 - ETA: 0s - loss: 0.6442 - accuracy: 0.60 - ETA: 0s - loss: 0.6397 - accuracy: 0.61 - ETA: 0s - loss: 0.6792 - accuracy: 0.59 - ETA: 0s - loss: 0.7110 - accuracy: 0.57 - ETA: 0s - loss: 0.7121 - accuracy: 0.59 - ETA: 0s - loss: 0.7188 - accuracy: 0.58 - ETA: 0s - loss: 0.7328 - accuracy: 0.58 - ETA: 0s - loss: 0.7194 - accuracy: 0.59 - ETA: 0s - loss: 0.7092 - accuracy: 0.59 - 1s 9ms/step - loss: 0.7063 - accuracy: 0.5989 - val_loss: 0.6847 - val_accuracy: 0.6037
Epoch 5/10
75/75 [==============================] - ETA: 0s - loss: 0.6567 - accuracy: 0.56 - ETA: 0s - loss: 0.6658 - accuracy: 0.57 - ETA: 0s - loss: 0.6878 - accuracy: 0.56 - ETA: 0s - loss: 0.6887 - accuracy: 0.58 - ETA: 0s - loss: 0.6808 - accuracy: 0.59 - ETA: 0s - loss: 0.6716 - accuracy: 0.59 - ETA: 0s - loss: 0.6712 - accuracy: 0.59 - ETA: 0s - loss: 0.6821 - accuracy: 0.58 - ETA: 0s - loss: 0.718

75/75 [==============================] - ETA: 0s - loss: 0.5774 - accuracy: 0.64 - ETA: 0s - loss: 0.6489 - accuracy: 0.63 - ETA: 0s - loss: 0.6533 - accuracy: 0.60 - ETA: 0s - loss: 0.6544 - accuracy: 0.60 - ETA: 0s - loss: 0.6538 - accuracy: 0.59 - ETA: 0s - loss: 0.6612 - accuracy: 0.59 - ETA: 0s - loss: 0.6618 - accuracy: 0.59 - ETA: 0s - loss: 0.6675 - accuracy: 0.57 - ETA: 0s - loss: 0.6705 - accuracy: 0.57 - ETA: 0s - loss: 0.6719 - accuracy: 0.57 - ETA: 0s - loss: 0.6734 - accuracy: 0.5600